In [1]:
import os
import numpy as np
import trimesh
import random

from scipy.spatial import cKDTree
from collections import Counter
from pyproj import CRS, Transformer
from scipy.spatial.distance import cosine

In [2]:
# ------------------------------- reading mesh file ---------------------------------
mesh_file = '/media/shangfeng/storage/shangfeng/BuildingWorld/Calgary/Buildings3D_2023January3/OID_1/esriGeometryMultiPatch.obj'

vertices = []
edges = []

with open(mesh_file, 'r') as f:
    for lines in f.readlines():
        if lines.startswith('v '):
            line = lines.strip().split(' ')
            vertices.append([np.float64(line[1]), np.float64(line[2]), np.float64(line[3])])
        elif lines.startswith('f '):
            line = lines.strip().split(' ')
            faces = np.array(line[1:], dtype=np.int32)
            edges.append([sorted((faces[i].tolist() - 1, faces[(i + 1) % len(faces)].tolist() - 1)) for i in range(len(faces))])
            # for i in range(len(faces)):
            #     edge = tuple(sorted((faces[i].tolist(), faces[(i + 1) % len(faces)].tolist())))
            #     edges.add(edge)
            # edges

vertices = np.array(vertices)

In [3]:
def are_planes_coplanar(ptsA, ptsB, angle_tol=1e-1, dist_tol=1e-1):
    # 1. 估计法向量和参考点
    nA, a1 = estimate_plane_normal(ptsA)
    nB, b1 = estimate_plane_normal(ptsB)

    # 2. 法向量是否平行
    if not np.allclose(np.cross(nA, nB), 0, atol=angle_tol):
        return False

    # 3. 点是否在另一平面上
    return abs(np.dot(nA, b1 - a1)) < dist_tol


def estimate_plane_normal(points):
    centroid = np.mean(points, axis=0)
    centered = points - centroid
    cov = np.cov(centered.T)
    eigvals, eigvecs = np.linalg.eigh(cov)
    normal = eigvecs[:, np.argmin(eigvals)]
    return normal / np.linalg.norm(normal), centroid


In [4]:
final_edges = []
while edges:
    edge = edges.pop(0)
    for e in edge:
        result = next((group for group in edges if e in group), None)
        if result is not None:
            first_face = list(dict.fromkeys([x for pair in edge for x in pair]))
            second_face = list(dict.fromkeys([x for pair in result for x in pair]))
            first_face = np.array([vertices[idx] for idx in first_face])
            second_face = np.array([vertices[idx] for idx in second_face])
            # print(first_face, second_face)
            if are_planes_coplanar(first_face, second_face):
                for r in result:
                    edge.append(r)
                while e in edge:
                    edge.remove(e)
                edges.remove(result)
                print(edge)

    final_edges.append(edge)

[[7, 8], [6, 7], [6, 9], [119, 120], [8, 119], [9, 120]]
[[49, 50], [47, 48], [47, 50], [49, 98], [97, 98], [48, 97]]
[[51, 61], [60, 61], [51, 59], [60, 145], [145, 146], [59, 146]]
[[69, 70], [68, 69], [67, 70], [68, 132], [132, 133], [67, 133]]
[[73, 74], [72, 73], [71, 74], [72, 116], [116, 118], [71, 118]]
[[83, 84], [82, 83], [81, 84], [82, 149], [149, 150], [81, 150]]


In [5]:
flat = [tuple(e) for edge in final_edges for e in edge]
unique = list(set(flat))
edges = [list(t) for t in unique]
print(edges)

edges = list(edges)
edges.sort()
edges = np.array(edges, dtype=np.int32)

[[55, 57], [72, 73], [105, 107], [43, 46], [21, 80], [44, 45], [33, 100], [99, 100], [15, 158], [64, 87], [8, 119], [87, 89], [152, 154], [45, 147], [12, 25], [59, 146], [102, 144], [34, 37], [83, 85], [132, 133], [14, 159], [123, 139], [137, 138], [168, 171], [46, 148], [24, 26], [41, 42], [96, 97], [37, 170], [18, 19], [73, 74], [67, 70], [23, 134], [13, 154], [121, 160], [116, 118], [4, 130], [126, 128], [14, 15], [9, 120], [5, 131], [111, 140], [48, 166], [170, 171], [2, 4], [122, 124], [68, 132], [53, 55], [15, 16], [13, 15], [7, 12], [47, 48], [25, 27], [35, 37], [90, 92], [107, 108], [47, 121], [162, 163], [20, 79], [134, 135], [93, 127], [38, 39], [31, 33], [103, 104], [5, 133], [29, 164], [75, 76], [58, 90], [85, 86], [72, 116], [26, 137], [47, 50], [64, 66], [41, 165], [1, 92], [1, 156], [50, 122], [96, 119], [65, 168], [6, 27], [48, 97], [22, 23], [51, 59], [38, 41], [77, 78], [93, 129], [49, 50], [114, 115], [75, 78], [20, 108], [109, 110], [13, 14], [95, 120], [151, 152], 

In [6]:
# --------------------------- Merge Vertices ---------------------------
# building tree
tree = cKDTree(vertices)
groups = tree.query_ball_tree(tree, 10e-2)
# print(groups)

# merge vertices
vertex_map = np.full(len(vertices), -1, dtype=int)
# print(vertex_map)
new_vertices = []
for i, group in enumerate(groups):
    if vertex_map[i] == -1:
        new_index = len(new_vertices)
        new_vertices.append(np.mean(vertices[group], axis=0))
        for idx in group:
            vertex_map[idx] = new_index

edges = np.array([[vertex_map[idx] for idx in edge] for edge in edges])
print(edges)
# new_faces = np.array([[vertex_map[idx] for idx in face] for face in faces])
# new_mesh.export("/media/shangfeng/storage/shangfeng/BuildingWorld/Calgary/wireframe/merged_model.obj")

[[  0   1]
 [  0   3]
 [  0  58]
 [  0  88]
 [  1   2]
 [  1  88]
 [  1  90]
 [  1  90]
 [  2   3]
 [  2   4]
 [  3   5]
 [  4   5]
 [  4 119]
 [  4 120]
 [  4  90]
 [  5 120]
 [  5 122]
 [  6   7]
 [  6   9]
 [  6  12]
 [  6  27]
 [  7   8]
 [  7  11]
 [  7  12]
 [  8 112]
 [  8 141]
 [  9 113]
 [ 10  11]
 [ 10  12]
 [ 11  12]
 [ 11 141]
 [ 12  25]
 [ 13  14]
 [ 13  15]
 [ 13  16]
 [ 13 140]
 [ 14  15]
 [ 14 139]
 [ 14  13]
 [ 14 143]
 [ 15  16]
 [ 15 142]
 [ 17  18]
 [ 17  20]
 [ 18  19]
 [ 18 102]
 [ 18 104]
 [ 18 145]
 [ 19  20]
 [ 19  21]
 [ 19  22]
 [ 20  21]
 [ 20  77]
 [ 20  17]
 [ 21  23]
 [ 21  78]
 [ 22  23]
 [ 22  24]
 [ 22  25]
 [ 23  24]
 [ 23 124]
 [ 24  26]
 [ 25  26]
 [ 25  27]
 [ 26  27]
 [ 26  24]
 [ 26 125]
 [ 27 126]
 [ 28  29]
 [ 28  31]
 [ 28  32]
 [ 29  30]
 [ 29 131]
 [ 29  97]
 [ 29 148]
 [ 30  31]
 [ 30 130]
 [ 30 131]
 [ 31  33]
 [ 32  33]
 [ 32  34]
 [ 32  35]
 [ 33  34]
 [ 33  96]
 [ 34  36]
 [ 34  37]
 [ 34  95]
 [ 35  36]
 [ 35  37]
 [ 35  38]
 [ 35 148]

In [7]:
vertices = new_vertices
print(len(edges))
edges = np.unique(edges, axis=0)
wireframe_vertices_index = edges.flatten()
wireframe_vertices_index = np.unique(wireframe_vertices_index)
print(len(edges))

288
275


In [15]:
# merge multiple segments in one line 
merged_edges = []
edges_list = edges.copy()
vertices_list = vertices.copy()
print('edges_list: ', edges_list)
# print('vertices_list: ', vertices_list)
# print(type(vertices))

for edge in edges_list:
    v1_idx, v2_idx = edge
    v1 = vertices_list[v1_idx]
    v2 = vertices_list[v2_idx]

    # check if the edge is connected to any other edges
    merged = False
    temp_merged_edges = merged_edges.copy()
    print('temp_merged_edges: ', temp_merged_edges)
    for other_edge in temp_merged_edges:
        o1_idx, o2_idx = other_edge
        if (v1_idx not in [o1_idx, o2_idx]) and (v2_idx not in [o1_idx, o2_idx]):
            continue
        o1 = vertices_list[o1_idx]
        o2 = vertices_list[o2_idx]

        d1 = np.array(v2) - np.array(v1)
        d2 = np.array(o2) - np.array(o1)
        value_1 = 1 - abs(1 - cosine(d1, d2))

        vs, count = np.unique(np.array([v1, v2, o1, o2]), axis=0, return_counts=True)
        vs = vs[count == 1]
        try:
            d3 = vs[0] - vs[1]
        except:
            merged = True
        value_2 = 1 - abs(1 - cosine(d3, d2))

        # if the angle between the two edges is small enough, merge them
        if (value_1 < 0.001) and (value_2 < 0.01):
            merged_edges.remove(other_edge)
            if v1_idx in [o1_idx, o2_idx]:
                if v1_idx == o1_idx:
                    merged_edges.append((v2_idx, o2_idx))
                    v1_idx = o2_idx
                elif v1_idx == o2_idx:
                    merged_edges.append((v2_idx, o1_idx))
                    v1_idx = o1_idx
                v1 = vertices[v1_idx]
            elif v2_idx in [o1_idx, o2_idx]:
                if v2_idx == o1_idx:
                    merged_edges.append((v1_idx, o2_idx))
                    v2_idx = o2_idx
                elif v2_idx == o2_idx:
                    merged_edges.append((v1_idx, o1_idx))
                    v2_idx = o1_idx
                v2 = vertices[v2_idx]
            merged = True

    if not merged:
        merged_edges.append(edge)

edges_list = merged_edges

# Remove duplicate edges
wireframe_edges = np.sort(edges_list, axis=1)
wireframe_edges = np.unique(wireframe_edges, axis=0)
print('Wireframe edges: ', len(wireframe_edges))

# Remove duplicate vertices
wireframe_vertices_index = wireframe_edges.flatten()
wireframe_vertices_index = np.unique(wireframe_vertices_index)
wireframe_vertices_index = np.sort(wireframe_vertices_index)
print('Wireframe vertices: ', len(wireframe_vertices_index))

edges_list:  [[  0   1]
 [  0   3]
 [  0  58]
 [  0  88]
 [  1   2]
 [  1  88]
 [  1  90]
 [  2   3]
 [  2   4]
 [  3   5]
 [  4   5]
 [  4  90]
 [  4 119]
 [  4 120]
 [  5 120]
 [  5 122]
 [  6   7]
 [  6   9]
 [  6  12]
 [  6  27]
 [  7   8]
 [  7  11]
 [  7  12]
 [  8 112]
 [  8 141]
 [  9 113]
 [ 10  11]
 [ 10  12]
 [ 11  12]
 [ 11 141]
 [ 12  25]
 [ 13  14]
 [ 13  15]
 [ 13  16]
 [ 13 140]
 [ 14  13]
 [ 14  15]
 [ 14 139]
 [ 14 143]
 [ 15  16]
 [ 15 142]
 [ 17  18]
 [ 17  20]
 [ 18  19]
 [ 18 102]
 [ 18 104]
 [ 18 145]
 [ 19  20]
 [ 19  21]
 [ 19  22]
 [ 20  17]
 [ 20  21]
 [ 20  77]
 [ 21  23]
 [ 21  78]
 [ 22  23]
 [ 22  24]
 [ 22  25]
 [ 23  24]
 [ 23 124]
 [ 24  26]
 [ 24 123]
 [ 25  26]
 [ 25  27]
 [ 26  24]
 [ 26  27]
 [ 26 125]
 [ 27 126]
 [ 28  29]
 [ 28  31]
 [ 28  32]
 [ 29  30]
 [ 29  97]
 [ 29 131]
 [ 29 148]
 [ 30  31]
 [ 30 130]
 [ 30 131]
 [ 31  33]
 [ 32  33]
 [ 32  34]
 [ 32  35]
 [ 33  34]
 [ 33  96]
 [ 34  36]
 [ 34  37]
 [ 34  95]
 [ 35  36]
 [ 35  37]
 [ 35  3

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [8]:
wireframe_file = '/media/shangfeng/storage/shangfeng/BuildingWorld/Calgary/wireframe/1.obj'
with open(wireframe_file, 'w') as f:
    f.write('# OBJ file\n')
    f.write('# Vertices: '+ str(len(wireframe_vertices_index)) +'\n')
    f.write('# Edges: '+ str(len(edges)) +'\n')
    
    for vertex_index in wireframe_vertices_index:
        f.write('v ')
        f.write(' '.join(map(str, vertices[int(vertex_index)])) + '\n')
        
    for edge in edges:
        f.write('l ')
        f.write(str(np.where(wireframe_vertices_index == edge[0])[0][0]+1) + ' ')
        f.write(str(np.where(wireframe_vertices_index == edge[1])[0][0]+1) + '\n')


In [9]:
print(wireframe_vertices_index)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154]


In [62]:
print(len(vertices))

115
